In [ ]:
import siibra

# Define region of interest map

Here we define a region of interest by annotation a point in BigBrain space, and building a heatmap volume of the point with a certain uncertainty.

In [ ]:
pt = siibra.from_json("""
{
  "@id": "2343bfd0",
  "@type": "https://openminds.ebrains.eu/sands/CoordinatePoint",
  "coordinateSpace": {
    "@id": "minds/core/referencespace/v1.0.0/a1655b99-82f1-420f-a3c2-fe80fd4c8588"
  },
  "coordinates": [
    {
      "@id": "81c7e0f0",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": -44.36846,
      "unit": {
        "@id": "id.link/mm"
      }
    },
    {
      "@id": "b8f02fa8",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": 4.32578,
      "unit": {
        "@id": "id.link/mm"
      }
    },
    {
      "@id": "10217fd0",
      "@type": "https://openminds.ebrains.eu/core/QuantitativeValue",
      "value": 30.56429,
      "unit": {
        "@id": "id.link/mm"
      }
    }
  ]
}
""")

In [ ]:
# convert the point to an image volume with uncertainty radius
pc = siibra.PointCloud([pt], space='bigbrain', sigma_mm=0.3)
query_volume = siibra.volumes.from_pointcloud(pc)

In [ ]:
# plot this query volume on top of a lower-resolution copy of BigBrain
import matplotlib.pyplot as plt
from nilearn import plotting
bigbrain = siibra.get_template('bigbrain').fetch()
f = plt.figure(figsize=(13, 4))
plotting.plot_stat_map(query_volume.fetch(), bg_img=bigbrain, cmap='viridis', figure=f)

# Sample cortical image patches inside query region

The volume can be used to let siibra sample cortical patches from BigBrain 1 micron data.
For this purpose, siibra samples vertices on the cortical midsurface close to the region of interest, and derives oriented patch bounding boxes using the layer surface geometry. 
The number of proposed patches depends on the size of the region of interest - larger heatmaps will result in more patches and longer runtimes for the query.
The query can take some time, since siibra resamples image data into oriented patches.
The resulting image objects come with spatial metadata that preserves their anchoring in the BigBrain space.

In [ ]:
# run feature query for cortical patch extraction.
# this can take a little, since suitable sample patches will be resampled into upright position.
features = siibra.features.get(query_volume, siibra.features.cellular.BigBrain1MicronPatch)

In [ ]:
# retrieve the first of the resulting patches.
assert len(features) > 0
patch = features[0]
patch_img = patch.fetch()

In [ ]:
# fetch layer mask for the patch
layermaps = siibra.get_map('layers', space='bigbrain')
voi = patch.get_boundingbox()
layermask = layermaps.fetch(fragment='left', format='image', voi=voi, resolution_mm=-1)

In [ ]:
# plot the patch in 3D context as well as the corresponding layer mask.
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,6))
plotting.plot_img(patch_img, bg_img=bigbrain, display_mode='tiled', axes=ax1)
plotting.plot_img(patch_img, bg_img=None, display_mode='y', cmap='gray', axes=ax2, cut_coords=[voi.minpoint[1]])
plotting.plot_img(layermask, bg_img=None, display_mode='y', cmap='Set1', axes=ax3, cut_coords=[voi.minpoint[1]])

# Use an AI model to detect cells in the patch

We use a pre-trained Contour Proposal Network (Eric Upschulte et al.) from celldetection.org to segment cell bodies in the resulting patch image.
For this, we reformat the patch into a 3-channel float image.
To reduce runtime and memory needs of this notebook, we limit the extraction to a subwindow in side the patch.

In [ ]:
# format into 3-channel 2D-image for cell detection
import numpy as np
img2D = patch_img.get_fdata().squeeze()[2100:2900, :500] / 2**16
img2D_3ch = np.stack((img2D,) * 3, 2)

## Variant 1: Run the model on HuggingFace

In [ ]:
# run the detection on HuggingFace.
# For this, store image to temporary file.
from gradio_client import Client, handle_file
client = Client("ericup/celldetection")
plt.imsave("tmp.png", (img2D_3ch * 255).astype('uint8'), )
result = client.predict(
    filename=handle_file("tmp.png"),
    model="vacumu_CpnResNeXt101UNet-f33b2634bb51f299",
    enable_score_threshold=False,
    enable_nms_threshold=False,
    enable_samples=False,
    use_label_channels=True,
    api_name="/predict"
)

In [ ]:
# Load overlay mask to visualize the result
overlay_filename, img_filename, h5_filename, csv_filename = result
overlay = plt.imread(overlay_filename)
plt.imshow(img2D_3ch)
plt.imshow(overlay)

## Variant 2: Run CPN model here

In [ ]:
import torch
import celldetection as cd

In [ ]:
# retrieve CPN model for cell detection
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = cd.fetch_model('vacumu_CpnResNeXt101UNet-f33b2634bb51f299').to(device)

In [ ]:
# run cell detection
x = cd.to_tensor(img2D_3ch, transpose=True, device=device, dtype=torch.float32)
x = x[None]
model.eval()
with torch.no_grad():
    y = model(x)

In [ ]:
# Show the contours
contours = y["contours"][0].cpu().data.numpy()
plt.figure()
plt.imshow(img2D, cmap='gray')
for c in contours:
    X, Y = c.T
    plt.plot(X, Y, '-', lw=1.5)

# Published cell densities

Siibra has access to published datasets with layerwise cell densities, based on a similar workflow. These consist of 10 patches each for a selection of cytoarchitectonic brain areas. We obtain a list of the available data features by querying with the whole parcellation. We plot the first of them to get an impression.

In [ ]:
with siibra.QUIET:
    features = siibra.features.get(
        siibra.parcellations.get('julich 3.1'), 
        siibra.features.cellular.LayerwiseCellDensity
    )

In [ ]:
# The feature contains a DataFrame with the measurements.
features[0].plot()
features[0].data